In [1]:
# require(data.table)
# require(bit64)
# require(dbscan)
# require(doParallel)
# require(rBayesianOptimization)
# path='../input/train_1/'
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

from trackml.dataset import load_event, load_dataset
from trackml.score import score_event
from trackml.randomize import shuffle_hits

from sklearn.preprocessing import StandardScaler
import hdbscan as _hdbscan
from scipy import stats
from tqdm import tqdm

import time

from sklearn.cluster.dbscan_ import dbscan
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

from sklearn.neighbors import KDTree
import hdbscan
from bayes_opt import BayesianOptimization

# https://www.ellicium.com/python-multiprocessing-pool-process/
# http://sebastianraschka.com/Articles/2014_multiprocessing.html
from multiprocessing import Pool

import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import time
import hdbscan as _hdbscan
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

In [13]:
# https://stackoverflow.com/questions/29246455/python-setting-decimal-place-range-without-rounding
import math
def truncate(f, n):
    return math.floor(f * 10 ** n) / 10 ** n

In [3]:
def create_one_event_submission(event_id, hits, labels):
    sub_data = np.column_stack(([event_id]*len(hits), hits.hit_id.values, labels))
    submission = pd.DataFrame(data=sub_data, columns=["event_id", "hit_id", "track_id"]).astype(int)
    return submission

In [4]:
def preprocess(hits):        
    x = hits.x.values
    y = hits.y.values
    z = hits.z.values

    r = np.sqrt(x**2 + y**2 + z**2)
    hits['x2'] = x/r
    hits['y2'] = y/r

    r = np.sqrt(x**2 + y**2)
    hits['z2'] = z/r

    ss = StandardScaler()
    X = ss.fit_transform(hits[['x2', 'y2', 'z2']].values)
#     for i, rz_scale in enumerate(self.rz_scales):
#         X[:,i] = X[:,i] * rz_scale

    return X

In [5]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

#------------------------------------------------------

def make_counts(labels):
    
    
    _,reverse,count = np.unique(labels,return_counts=True,return_inverse=True)
    counts = count[reverse]
    counts[labels==0]=0
    
    return counts

def one_loop(param):
    
    # <todo> tune your parameters or design your own features here! 
    
    i,m, x,y,z, d,r, a, a_start,a_step = param
    #print('\r %3d  %+0.8f '%(i,da), end='', flush=True)
    
    da = m*(a_start - (i*a_step))
    aa = a + np.sign(z)*z*da 
    zr = z/r
    
    X = StandardScaler().fit_transform(np.column_stack([aa, aa/zr, zr, 1/zr, aa/zr + 1/zr]))
    _,l = dbscan(X, eps=0.0035, min_samples=1,)

    return l

def one_loop1(param):
    
    # <todo> tune your parameters or design your own features here! 
    
    i,m, x,y,z, d,r,r2,z2,a, a_start,a_step = param
    #print('\r %3d  %+0.8f '%(i,da), end='', flush=True)
    
    da = m*(a_start - (i*a_step))
    aa = a + np.sign(z)*z*da
#     if m == 1:
#         print(da)
    zr = z/r # this is cot(theta), 1/zr is tan(theta)
    theta = np.arctan2(r, z)
    ct = np.cos(theta)
    st = np.sin(theta)
    tt = np.tan(theta)
#     ctt = np.cot(theta)
    z2r = z2/r
    z2r2 = z2/r2
#     X = StandardScaler().fit_transform(df[['r2', 'theta_1', 'dip_angle', 'z2', 'z2_1', 'z2_2']].values)

    caa = np.cos(aa)
    saa = np.sin(aa)
    taa = np.tan(aa)
    ctaa = 1/taa
    
#     0.000005
    deps = 0.0000025
    X = StandardScaler().fit_transform(np.column_stack([caa, saa, tt, 1/tt]))
    l= DBSCAN(eps=0.0035+i*deps,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_
#     _,l = dbscan(X, eps=0.0035, min_samples=1,algorithm='auto')
    
    return l

def one_loop2(param):
    
    # <todo> tune your parameters or design your own features here! 
    
    i,m, x,y,z, d,r,r2,z2,a, a_start,a_step = param
    #print('\r %3d  %+0.8f '%(i,da), end='', flush=True)
    
    da = m*(a_start - (i*a_step))
    aa = a + np.sign(z)*z*da
#     if m == 1:
#         print(da)
    zr = z/r # this is cot(theta), 1/zr is tan(theta)
    theta = np.arctan2(r, z)
    ct = np.cos(theta)
    st = np.sin(theta)
    tt = np.tan(theta)
#     ctt = np.cot(theta)
    z2r = z2/r
    z2r2 = z2/r2
#     X = StandardScaler().fit_transform(df[['r2', 'theta_1', 'dip_angle', 'z2', 'z2_1', 'z2_2']].values)

    caa = np.cos(aa)
    saa = np.sin(aa)
    taa = np.tan(aa)
    ctaa = 1/taa
    
#     0.000005
    deps = 0.0000025
    X = StandardScaler().fit_transform(np.column_stack([caa, saa, tt, 1/tt]))
    l= DBSCAN(eps=0.0035+i*deps,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_
#     _,l = dbscan(X, eps=0.0035, min_samples=1,algorithm='auto')
    
    return l

def do_dbscan_predict(df):
    
    x  = df.x.values
    y  = df.y.values
    z  = df.z.values
    r  = np.sqrt(x**2+y**2)
    d  = np.sqrt(x**2+y**2+z**2)
    a  = np.arctan2(y,x)

    x2 = df['x']/d
    y2 = df['y']/d
    z2 = df['z']/r

    r2 = np.sqrt(x2**2 + y2**2)
    phi = np.arctan2(y, x)
    phi_deg= np.degrees(np.arctan2(y, x))
    phi2 = np.arctan2(y2, x2)
    phi2_deg = np.degrees(np.arctan2(y2, x2))
    
    
    scores = []

    a_start,a_step,a_num = 0.00100,0.0000095,150

    params  = [(i,m, x,y,z,d,r,r2,z2, a, a_start,a_step) for i in range(a_num) for m in [-1,1]]

    if 1: 
        pool = Pool(processes=1)
        ls   = pool.map( one_loop1, params )


    if 0:
        ls = [ one_loop(param) for param in  params ]


    ##------------------------------------------------

    num_hits=len(df)
    labels = np.zeros(num_hits,np.int32)
    counts = np.zeros(num_hits,np.int32)
    for l in ls:
        c = make_counts(l)
        idx = np.where((c-counts>0) & (c<20))[0]
        labels[idx] = l[idx] + labels.max()
        counts = make_counts(labels)
       

#     cl = hdbscan.HDBSCAN(min_samples=1,min_cluster_size=7,
#                          metric='braycurtis',cluster_selection_method='leaf',algorithm='best', 
#                          leaf_size=50)
    
#     X = preprocess(df)
#     l1 = pd.Series(labels)
#     labels = np.unique(l1)
   
# #   print(X.shape)
# #   print(len(labels_org))
# #   print(len(labels_org[labels_org ==0]))
# #   print(len(labels_org[labels_org ==-1]))
    
#     n_labels = 0
#     while n_labels < len(labels):
#         n_labels = len(labels)
#         max_len = np.max(l1)
#         s = list(l1[l1 == 0].keys())
#         X = X[s]
#         print(X.shape)
#         if X.shape[0] <= 1:
#             break
#         l = cl.fit_predict(X)+max_len
# #         print(len(l))
#         l1[l1 == 0] = l
#         labels = np.unique(l1)
    
    return labels

## reference----------------------------------------------
def do_dbscan0_predict(df):
    x = df.x.values
    y = df.y.values
    z = df.z.values
    r = np.sqrt(x**2+y**2)
    d = np.sqrt(x**2+y**2+z**2)

    X = StandardScaler().fit_transform(np.column_stack([
        x/d, y/d, z/r]))
    _,labels = dbscan(X,
                eps=0.0075,
                min_samples=1,
                algorithm='auto',
                n_jobs=-1)

    #labels = hdbscan(X, min_samples=1, min_cluster_size=5, cluster_selection_method='eom')

    return labels

## reference----------------------------------------------
def do_dbscan0_predict(df):
    x = df.x.values
    y = df.y.values
    z = df.z.values
    r = np.sqrt(x**2+y**2)
    d = np.sqrt(x**2+y**2+z**2)

    X = StandardScaler().fit_transform(np.column_stack([
        x/d, y/d, z/r]))
    _,labels = dbscan(X,
                eps=0.0075,
                min_samples=1,
                algorithm='auto',
                n_jobs=-1)

    #labels = hdbscan(X, min_samples=1, min_cluster_size=5, cluster_selection_method='eom')

    return labels
var1 = 0
def extend(submission,hits):
    df = submission.merge(hits,  on=['hit_id'], how='left')
#     if var1 != 0:
#         df = submission.merge(hits,  on=['hit_id'], how='left')
#     else:
#         df = hits
#     df = submission.append(hits)
#     print(df.head())
    df = df.assign(d = np.sqrt( df.x**2 + df.y**2 + df.z**2 ))
    df = df.assign(r = np.sqrt( df.x**2 + df.y**2))
    df = df.assign(arctan2 = np.arctan2(df.z, df.r))

    for angle in range(-180,180,1):

        print ('\r %f'%angle, end='',flush=True)
        #df1 = df.loc[(df.arctan2>(angle-0.5)/180*np.pi) & (df.arctan2<(angle+0.5)/180*np.pi)]
        df1 = df.loc[(df.arctan2>(angle-1.0)/180*np.pi) & (df.arctan2<(angle+1.0)/180*np.pi)]

#         if len(df1) == 0:
#             continue
        min_num_neighbours = len(df1)
        if min_num_neighbours<4: continue

        hit_ids = df1.hit_id.values
        x,y,z = df1.as_matrix(columns=['x', 'y', 'z']).T
        r  = (x**2 + y**2)**0.5
        r  = r/1000
        a  = np.arctan2(y,x)
        tree = KDTree(np.column_stack([a,r]), metric='euclidean')

#         print(df1.head())
        track_ids = list(df1.track_id.unique())
        num_track_ids = len(track_ids)
        min_length=3

        for i in range(num_track_ids):
            p = track_ids[i]
            if p==0: continue

            idx = np.where(df1.track_id==p)[0]
            if len(idx)<min_length: continue

            if angle>0:
                idx = idx[np.argsort( z[idx])]
            else:
                idx = idx[np.argsort(-z[idx])]


            ## start and end points  ##
            idx0,idx1 = idx[0],idx[-1]
            a0 = a[idx0]
            a1 = a[idx1]
            r0 = r[idx0]
            r1 = r[idx1]

            da0 = a[idx[1]] - a[idx[0]]  #direction
            dr0 = r[idx[1]] - r[idx[0]]
            direction0 = np.arctan2(dr0,da0) 

            da1 = a[idx[-1]] - a[idx[-2]]
            dr1 = r[idx[-1]] - r[idx[-2]]
            direction1 = np.arctan2(dr1,da1) 


            ## extend start point
            ns = tree.query([[a0,r0]], k=min(20,min_num_neighbours), return_distance=False)
            ns = np.concatenate(ns)
            direction = np.arctan2(r0-r[ns],a0-a[ns])
            ns = ns[(r0-r[ns]>0.01) &(np.fabs(direction-direction0)<0.04)]

            for n in ns:
                df.loc[ df.hit_id==hit_ids[n],'track_id' ] = p 

            ## extend end point
            ns = tree.query([[a1,r1]], k=min(20,min_num_neighbours), return_distance=False)
            ns = np.concatenate(ns)

            direction = np.arctan2(r[ns]-r1,a[ns]-a1)
            ns = ns[(r[ns]-r1>0.01) &(np.fabs(direction-direction1)<0.04)] 

            for n in ns:
                df.loc[ df.hit_id==hit_ids[n],'track_id' ] = p
    #print ('\r')
#     df = df[['particle_id', 'weight', 'event_id', 'hit_id', 'track_id']]
    df = df[['event_id', 'hit_id', 'track_id']]
    return df

In [6]:
rz_scales=[0.65, 0.965, 1.528]
def _eliminate_outliers(labels,M):
        norms=np.zeros((len(labels)),np.float32)
        indices=np.zeros((len(labels)),np.float32)
        for i, cluster in tqdm(enumerate(labels),total=len(labels)):
            if cluster == 0:
                continue
            index = np.argwhere(self.clusters==cluster)
            index = np.reshape(index,(index.shape[0]))
            indices[i] = len(index)
            x = M[index]
            norms[i] = self._test_quadric(x)
        threshold1 = np.percentile(norms,90)*5
        threshold2 = 25
        threshold3 = 6
        for i, cluster in enumerate(labels):
            if norms[i] > threshold1 or indices[i] > threshold2 or indices[i] < threshold3:
                self.clusters[self.clusters==cluster]=0

def _test_quadric(x):
    if x.size == 0 or len(x.shape)<2:
        return 0
    Z = np.zeros((x.shape[0],10), np.float32)
    Z[:,0] = x[:,0]**2
    Z[:,1] = 2*x[:,0]*x[:,1]
    Z[:,2] = 2*x[:,0]*x[:,2]
    Z[:,3] = 2*x[:,0]
    Z[:,4] = x[:,1]**2
    Z[:,5] = 2*x[:,1]*x[:,2]
    Z[:,6] = 2*x[:,1]
    Z[:,7] = x[:,2]**2
    Z[:,8] = 2*x[:,2]
    Z[:,9] = 1
    v, s, t = np.linalg.svd(Z,full_matrices=False)        
    smallest_index = np.argmin(np.array(s))
    T = np.array(t)
    T = T[smallest_index,:]        
    norm = np.linalg.norm(np.dot(Z,T), ord=2)**2
    return norm

def _preprocess(hits):
    x = hits.x.values
    y = hits.y.values
    z = hits.z.values

    r = np.sqrt(x**2 + y**2 + z**2)
    hits['x2'] = x/r
    hits['y2'] = y/r

    r = np.sqrt(x**2 + y**2)
    hits['z2'] = z/r

    ss = StandardScaler()
    X = ss.fit_transform(hits[['x2', 'y2', 'z2']].values)
    for i, rz_scale in enumerate(self.rz_scales):
        X[:,i] = X[:,i] * rz_scale

    return X

In [7]:
def extend(submission,hits):
    df = submission.merge(hits,  on=['hit_id'], how='left')
#     df = submission.append(hits)
#     print(df.head())
    df = df.assign(d = np.sqrt( df.x**2 + df.y**2 + df.z**2 ))
    df = df.assign(r = np.sqrt( df.x**2 + df.y**2))
    df = df.assign(arctan2 = np.arctan2(df.z, df.r))

    for angle in range(-180,180,1):

        print ('\r %f'%angle, end='',flush=True)
        #df1 = df.loc[(df.arctan2>(angle-0.5)/180*np.pi) & (df.arctan2<(angle+0.5)/180*np.pi)]
        df1 = df.loc[(df.arctan2>(angle-1.0)/180*np.pi) & (df.arctan2<(angle+1.0)/180*np.pi)]

        min_num_neighbours = len(df1)
        if min_num_neighbours<4: continue

        hit_ids = df1.hit_id.values
        x,y,z = df1.as_matrix(columns=['x', 'y', 'z']).T
        r  = (x**2 + y**2)**0.5
        r  = r/1000
        a  = np.arctan2(y,x)
        tree = KDTree(np.column_stack([a,r]), metric='euclidean')

        track_ids = list(df1.track_id.unique())
        num_track_ids = len(track_ids)
        min_length=3

        for i in range(num_track_ids):
            p = track_ids[i]
            if p==0: continue

            idx = np.where(df1.track_id==p)[0]
            if len(idx)<min_length: continue

            if angle>0:
                idx = idx[np.argsort( z[idx])]
            else:
                idx = idx[np.argsort(-z[idx])]


            ## start and end points  ##
            idx0,idx1 = idx[0],idx[-1]
            a0 = a[idx0]
            a1 = a[idx1]
            r0 = r[idx0]
            r1 = r[idx1]

            da0 = a[idx[1]] - a[idx[0]]  #direction
            dr0 = r[idx[1]] - r[idx[0]]
            direction0 = np.arctan2(dr0,da0) 

            da1 = a[idx[-1]] - a[idx[-2]]
            dr1 = r[idx[-1]] - r[idx[-2]]
            direction1 = np.arctan2(dr1,da1) 


            ## extend start point
            ns = tree.query([[a0,r0]], k=min(20,min_num_neighbours), return_distance=False)
            ns = np.concatenate(ns)
            direction = np.arctan2(r0-r[ns],a0-a[ns])
            ns = ns[(r0-r[ns]>0.01) &(np.fabs(direction-direction0)<0.04)]

            for n in ns:
                df.loc[ df.hit_id==hit_ids[n],'track_id' ] = p 

            ## extend end point
            ns = tree.query([[a1,r1]], k=min(20,min_num_neighbours), return_distance=False)
            ns = np.concatenate(ns)

            direction = np.arctan2(r[ns]-r1,a[ns]-a1)
            ns = ns[(r[ns]-r1>0.01) &(np.fabs(direction-direction1)<0.04)] 

            for n in ns:
                df.loc[ df.hit_id==hit_ids[n],'track_id' ] = p
    #print ('\r')
#     df = df[['particle_id', 'weight', 'event_id', 'hit_id', 'track_id']]
    df = df[['event_id', 'hit_id', 'track_id']]
    return df

In [14]:
def merge(cl1, cl2): # merge cluster 2 to cluster 1
    d = pd.DataFrame(data={'s1':cl1,'s2':cl2})
    d['N1'] = d.groupby('s1')['s1'].transform('count')
    d['N2'] = d.groupby('s2')['s2'].transform('count')
    maxs1 = d['s1'].max()
    cond = np.where((d['N2'].values>d['N1'].values) & (d['N2'].values<25)) # tìm vị trí hit với nhit của cluster mới > nhits cluster cũ
    s1 = d['s1'].values 
    s1[cond] = d['s2'].values[cond]+maxs1 # gán tất cả các hit đó thuộc về track mới (+maxs1 để tăng label cho track để nó khác ban đầu)
    return s1

def extract_good_hits(truth, submission):
    tru = truth[['hit_id', 'particle_id', 'weight']].merge(submission, how='left', on='hit_id')
    tru['count_both'] = tru.groupby(['track_id', 'particle_id']).hit_id.transform('count')    
    tru['count_particle'] = tru.groupby(['particle_id']).hit_id.transform('count')
    tru['count_track'] = tru.groupby(['track_id']).hit_id.transform('count')
    return tru[(tru.count_both > 0.5*tru.count_particle) & (tru.count_both > 0.5*tru.count_track)]

def fast_score(good_hits_df):
    return good_hits_df.weight.sum()


def analyze_truth_perspective(truth, submission):
    tru = truth[['hit_id', 'particle_id', 'weight']].merge(submission, how='left', on='hit_id')
    tru['count_both'] = tru.groupby(['track_id', 'particle_id']).hit_id.transform('count')    
    tru['count_particle'] = tru.groupby(['particle_id']).hit_id.transform('count')
    tru['count_track'] = tru.groupby(['track_id']).hit_id.transform('count')
    good_hits = tru[(tru.count_both > 0.5*tru.count_particle) & (tru.count_both > 0.5*tru.count_track)]
    score = good_hits.weight.sum()
    
    anatru = tru.particle_id.value_counts().value_counts().sort_index().to_frame().rename({'particle_id':'true_particle_counts'}, axis=1)
    #anatru['true_particle_ratio'] = anatru['true_particle_counts'].values*100/np.sum(anatru['true_particle_counts'])

    anatru['good_tracks_counts'] = np.zeros(len(anatru)).astype(int)
    anatru['good_tracks_intersect_nhits_avg'] = np.zeros(len(anatru))
    anatru['best_detect_intersect_nhits_avg'] = np.zeros(len(anatru))
    for nhit in tqdm(range(4,20)):
        particle_list  = tru[(tru.count_particle==nhit)].particle_id.unique()
        intersect_count = 0
        good_tracks_count = 0
        good_tracks_intersect = 0
        for p in particle_list:
            nhit_intersect = tru[tru.particle_id==p].count_both.max()
            intersect_count += nhit_intersect
            corresponding_track = tru.loc[tru[tru.particle_id==p].count_both.idxmax()].track_id
            leng_corresponding_track = len(tru[tru.track_id == corresponding_track])
            
            if (nhit_intersect >= nhit/2) and (nhit_intersect >= leng_corresponding_track/2):
                good_tracks_count += 1
                good_tracks_intersect += nhit_intersect
        intersect_count = intersect_count/len(particle_list)
        anatru.at[nhit,'best_detect_intersect_nhits_avg'] = intersect_count
        anatru.at[nhit,'good_tracks_counts'] = good_tracks_count
        if good_tracks_count > 0:
            anatru.at[nhit,'good_tracks_intersect_nhits_avg'] = good_tracks_intersect/good_tracks_count
    
    return score, anatru, good_hits

def precision(truth, submission,min_hits):
    tru = truth[['hit_id', 'particle_id', 'weight']].merge(submission, how='left', on='hit_id')
    tru['count_both'] = tru.groupby(['track_id', 'particle_id']).hit_id.transform('count')    
    tru['count_particle'] = tru.groupby(['particle_id']).hit_id.transform('count')
    tru['count_track'] = tru.groupby(['track_id']).hit_id.transform('count')
    #print('Analyzing predictions...')
    predicted_list  = tru[(tru.count_track>=min_hits)].track_id.unique()
    good_tracks_count = 0
    ghost_tracks_count = 0
    fp_weights = 0
    tp_weights = 0
    for t in predicted_list:
        nhit_track = tru[tru.track_id==t].count_track.iloc[0]
        nhit_intersect = tru[tru.track_id==t].count_both.max()
        corresponding_particle = tru.loc[tru[tru.track_id==t].count_both.idxmax()].particle_id
        leng_corresponding_particle = len(tru[tru.particle_id == corresponding_particle])
        if (nhit_intersect >= nhit_track/2) and (nhit_intersect >= leng_corresponding_particle/2): #if the predicted track is good
            good_tracks_count += 1
            tp_weights += tru[(tru.track_id==t)&(tru.particle_id==corresponding_particle)].weight.sum()
            fp_weights += tru[(tru.track_id==t)&(tru.particle_id!=corresponding_particle)].weight.sum()
        else: # if the predicted track is bad
                ghost_tracks_count += 1
                fp_weights += tru[(tru.track_id==t)].weight.sum()
    all_weights = tru[(tru.count_track>=min_hits)].weight.sum()
    precision = tp_weights/all_weights*100
    print('Precision: ',precision,', good tracks:', good_tracks_count,', total tracks:',len(predicted_list),
           ', loss:', fp_weights, ', reco:', tp_weights, 'reco/loss', tp_weights/fp_weights)
    return precision


class Clusterer(object):
    def __init__(self):                        
        self.abc = []
          
    def initialize(self,dfhits):
        self.cluster = range(len(dfhits))
        
    def Hough_clustering(self,dfh,coef,epsilon,min_samples=1,n_loop=180,verbose=True): # [phi_coef,phi_coef,zdivrt_coef,zdivr_coef,xdivr_coef,ydivr_coef]
        merged_cluster = self.cluster
        mm = 1
        stepii = 0.000005
        count_ii = 0
        adaptive_eps_coefficient = 1
        
        for ii in np.arange(0, n_loop*stepii, stepii):
            count_ii += 1
            for jj in range(2):
                mm = mm*(-1)
                eps_new = epsilon + count_ii*adaptive_eps_coefficient*10**(-5)
                dfh['a1'] = dfh['a0'].values - np.nan_to_num(np.arccos(mm*ii*dfh['rt'].values))
                dfh['sina1'] = np.sin(dfh['a1'].values)
                dfh['cosa1'] = np.cos(dfh['a1'].values)
                dfh['ia1'] = np.nan_to_num(((mm*ii*dfh['rt'].values) * np.nan_to_num(np.arccos(mm*ii*dfh['rt'].values))) - (np.sqrt(1-(mm*ii*dfh['rt'].values)**2)))
                ss = StandardScaler()
                dfs = ss.fit_transform(dfh[['sina1','cosa1','zdivrt','zdivr','xdivr','ydivr','ia1', 'fundu', 'mom2']].values) 
                #dfs = scale_ignore_nan(dfh[['sina1','cosa1','zdivrt','zdivr','xdivr','ydivr']])
                dfs = np.multiply(dfs, coef)
                new_cluster=DBSCAN(eps=eps_new,min_samples=min_samples,metric='euclidean',n_jobs=4).fit(dfs).labels_
                merged_cluster = merge(merged_cluster, new_cluster)
                if verbose == True:
                    sub = create_one_event_submission(0, hits, merged_cluster)
                    good_hits = extract_good_hits(truth, sub)
                    score_1 = fast_score(good_hits)
#                     print('2r0_inverse:', ii*mm ,'. Score:', score_1)
                    #clear_output(wait=True)
        self.cluster = merged_cluster

def create_one_event_submission(event_id, hits, labels):
    sub_data = np.column_stack(([event_id]*len(hits), hits.hit_id.values, labels))
    submission = pd.DataFrame(data=sub_data, columns=["event_id", "hit_id", "track_id"]).astype(int)
    return submission  

def preprocess_hits(h,dz, theta0):
    y = h['y'].values
    x = h['x'].values
    h['zz'] =  h['z'].values + dz
    z = h['zz'].values
    h['r'] = np.sqrt(h['x'].values**2+h['y'].values**2+h['zz'].values**2)
#     r = h['r'].values
    h['rt'] = np.sqrt(h['x'].values**2+h['y'].values**2)
    R = h['r'].values
    h['a0'] = np.arctan2(h['y'].values,h['x'].values)
    h['zdivrt'] = h['zz'].values/h['rt'].values
    h['zdivr'] = h['zz'].values/h['r'].values
    h['xdivr'] = h['x'].values / h['r'].values
    h['ydivr'] = h['y'].values / h['r'].values
    h['fundu'] = np.arcsin((y * np.sin(theta0) - x * np.cos(theta0)) / R) / (z - dz)
#     h['mom2'] = np.sqrt(1 + ((z/R)**2)).round(1)
    zR = z/R
    h['mom2'] = [truncate(np.sqrt(1 + (i**2)),4) for i in zR]
    return h

In [9]:
# def get_hits():

data_dir = '../data/train'

#     event_ids = [
#             '000001030',##
#             '000001025','000001026','000001027','000001028','000001029',
#     ]

event_ids = [
        '000001030',##

]

sum=0
sum_score=0
for i,event_id in enumerate(event_ids):
    particles = pd.read_csv(data_dir + '/event%s-particles.csv'%event_id)
    hits  = pd.read_csv(data_dir + '/event%s-hits.csv'%event_id)
    cells = pd.read_csv(data_dir + '/event%s-cells.csv'%event_id)
    truth = pd.read_csv(data_dir + '/event%s-truth.csv'%event_id)
    particles = pd.read_csv(data_dir + '/event%s-particles.csv'%event_id)

    truth = pd.merge(truth, particles, how='left', on='particle_id')
    hits = pd.merge(hits, truth, how='left', on='hit_id')



In [10]:
# def get_hits():

data_dir = '../data/train'

#     event_ids = [
#             '000001030',##
#             '000001025','000001026','000001027','000001028','000001029',
#     ]

event_ids = [
        '000001030',##

]

sum=0
sum_score=0
def get_hits():
    for i,event_id in enumerate(event_ids):
        particles = pd.read_csv(data_dir + '/event%s-particles.csv'%event_id)
        hits  = pd.read_csv(data_dir + '/event%s-hits.csv'%event_id)
        cells = pd.read_csv(data_dir + '/event%s-cells.csv'%event_id)
        truth = pd.read_csv(data_dir + '/event%s-truth.csv'%event_id)
        particles = pd.read_csv(data_dir + '/event%s-particles.csv'%event_id)

        truth = pd.merge(truth, particles, how='left', on='particle_id')
        hits = pd.merge(hits, truth, how='left', on='hit_id')
        return hits


In [9]:


w1 = [0.9, 1.2]
w2 = [0.3, 0.7]
w3 = [0.1, 0.4]
Niter = [140, 190]


#     bo = BayesianOptimization(Fun4BO,pbounds = {'w1':w1,'w2':w2,'w3':w3,'Niter':Niter})
#     bo.maximize(init_points = 3, n_iter = 20, acq = "ucb", kappa = 2.576)
w1 = 1.1932215111905984
w2 = 0.39740553885387364
w3 = 0.3512647720585538
w4 = 0.1470
w5 = 0.2691
w6 = 0.0020
w6 = [0.0000001, 1.2] 
w10 = [0.00001, 1.2] 
w12 = [0.00001, 0.1] 
Niter = 179
w12 = 0.00045
# Fun4BO21(w12)
#     for w12 in np.arange(0.0002, 0.003, 0.00005):
#         print(w12)
#         Fun4BO21(w12)
#     bo = BayesianOptimization(Fun4BO21,pbounds = {'w12':w12})
#     bo.maximize(init_points = 20, n_iter = 5, acq = "ucb", kappa = 2.576)

# z1+z2: 13 | 05m39s |    0.55616 |    0.1124 | 
# z1 * z2: 13 | 05m40s |    0.55637 |    0.0404 | 
# tt = cos(theta), theta = np.arctan2(rt,z): 4 | 06m01s |    0.55196 |    0.2711 | 
# tt, theta = np.arctan2(rt2,z2);  8 | 05m39s |    0.55604 |    0.0005 | 
# cos + sin: 15 | 05m39s |    0.55714 |    0.2691 | 
# cos-sin: 9 | 05m51s |    0.55714 |    0.0020 | 
# cos+sin/cos-sin: 8 | 06m03s |    0.55694 |    0.0012 | 
# ctt, stt, ctt+stt/ctt-stt: 8 | 06m03s |    0.55273 |    0.0000 | 
# ctt: 10 | 05m39s |    0.55613 |    0.0047 | 
# caa * stt: 7 | 05m38s |    0.55613 |    0.0022 |
# saa * ctt: 15 | 05m38s |    0.55613 |    0.0033 |
# c1 = caa*stt, c2 = saa*ctt, c1+c2: 10 | 05m43s |    0.55622 |    0.0571 | 
# caa * saa: 7 | 05m45s |    0.55639 |    0.1548 | 
# caa/saa: 13 | 05m43s |    0.55613 |    0.000001 | 
# xx1, xx2: 12 | 06m04s |    0.55613 |    0.000001 |    0.000001 | 

# hdbscan - braycurtis: 0.14424123982821765
# hdbscan - euclidean : 0.1311

# eps: 0.0040: 0.57549
# tt: 21 | 07m08s |    0.57208 |    0.0000 |  (1e-07)
# ctt, stt (after new eqn: 25 | 07m33s |    0.57254 |    0.0026 | 
# ctt, stt: 0.00261: 0.5727074941034839
# caa, saa, z1, z2, rt/r, x/r, z3, ctt, stt, y1, y3:
# w1 = 1.1932215111905984, w2 = 0.39740553885387364, w3 = 0.3512647720585538, w4 = 0.1470, w5 = 0.01201
# w6 = 0.0205, w7 = 0.00261, w8 = 0.0049, w9 = 0.0012 (0.5717942069958433)
# 
# X = StandardScaler().fit_transform(np.column_stack([caa, saa, z1, z2, rt/r, x/r, y/r, z3, y1, y3])):
# w1 = 1.1932215111905984 w2 = 0.39740553885387364 w3 = 0.3512647720585538 w4 = 0.1470 w5 = 0.01201 w6 = 0.0003864 
# w7 = 0.0205 w8 = 0.0049 w9 = 0.00121 (0.57343)
# ctt, stt: 13 | 08m16s |    0.57343 |    0.0003 | (0.00032967312140735677)
# ctt, stt: 21 | 08m20s |    0.57343 |    0.0000 |  (1.4930496676654575e-05)
# ctt, stt: 15 | 08m14s |    0.57351 |    0.8435 | (t1 = theta+mm*(rt+ w11*rt**2)/1000*(ii/2)/180*np.pi)
# z4: 0.0245 (0.5735925042985041)
# # z4 0.0318 (0.5736635664313068)
# x4: 0.00001 (0.5735421714482896)
# x4: 0.00025 (0.5736999491677117)
# x4: 0.00045 (0.5737240529228144)

In [10]:
# Clustering by varying 
#model = Clusterer()
#model.initialize(hits) 
# c = [1.5,1.5,0.73,0.17,0.027,0.027, 1.] #[phi_coef,phi_coef,zdivrt_coef,zdivr_coef,xdivr_coef,ydivr_coef]
# min_samples_in_cluster = 1

# model = Clusterer()
# model.initialize(hits) 
# hits_with_dz = preprocess_hits(hits, 0)
# model.Hough_clustering(hits_with_dz,coef=c,epsilon=0.0048,min_samples=min_samples_in_cluster,
#                        n_loop=370,verbose=True)

# submission = create_one_event_submission(0, hits, model.cluster)
# print('\n')

# score = score_event(truth, submission)
# print(score)

In [11]:
def make_counts(labels):
    _,reverse,count = np.unique(labels,return_counts=True,return_inverse=True)
    counts = count[reverse]
    counts[labels==0]=0
    
    return counts

def Fun4BO11(w):
    hits = get_hits()
    print(w)
#     c = [w1,w1,w2,w3,w4,w5]
    dzs = list(np.linspace(-5.5, 5.5, 10))
    t = list(np.linspace(-np.pi, np.pi, 12))
    ls = []
    dz = 0
#     for dz in dzs:
    
    c = [1.77,1.77,0.734,0.14,0.0195,0.025,0.0008,0.0001,w]
    model = Clusterer()
    model.initialize(hits) 
    hits_with_dz = preprocess_hits(hits, dz, 0)
    min_samples_in_cluster = 1
    model.Hough_clustering(hits_with_dz,coef=c,epsilon=0.0048,min_samples=min_samples_in_cluster,
                   n_loop=370,verbose=True)

    ls.append(model.cluster)
    return model.cluster
    num_hits=len(hits)
    labels = np.zeros(num_hits,np.int32)
    counts = np.zeros(num_hits,np.int32)
    lss = []
    for l in ls:
        c = make_counts(l)
        idx = np.where((c-counts>0) & (c<20))[0]
        labels[idx] = l[idx] + labels.max()
        counts = make_counts(labels)
    return labels

In [32]:
%%time
labels = Fun4BO11()
    
submission = create_one_event_submission(0, hits, labels)
print('-------------------------')

score = score_event(truth, submission)
print(score)

# 20 z_shift: 0.5992258787927022, with extn: 0.6252801561124434

TypeError: Fun4BO11() missing 1 required positional argument: 'w'

In [15]:
sum_score=0
sum = 0
submission = pd.DataFrame(columns=['event_id', 'hit_id', 'track_id'],
    data=np.column_stack(([int(event_id),]*len(hits), hits.hit_id.values, labels))
).astype(int)


for i in range(8):
    submission = extend(submission,hits)
    score = score_event(truth, submission)
    print('[%2d] score : %0.8f'%(i, score))
    sum_score += score
    sum += 1

print('--------------------------------------')
sc = sum_score/sum
print(sc)

 179.0000000[ 0] score : 0.62077650
 179.0000000[ 1] score : 0.62456689
 179.0000000[ 2] score : 0.62482064
 179.0000000[ 3] score : 0.62586284
 179.0000000[ 4] score : 0.62595311
 179.0000000[ 5] score : 0.62689351
 179.0000000[ 6] score : 0.62637153
 179.0000000[ 7] score : 0.62699622
--------------------------------------
0.6252801561124434


In [15]:
for w in [1., 0.1, 0.01, 0.001, 0.0001, 0.00001]:
    print(w)
    labels = Fun4BO11(w)
    
    submission = create_one_event_submission(0, hits, labels)
    print('-------------------------')

    score = score_event(truth, submission)
    print(score)
# w is derivative of arccos
# 0.01: 0.48149631563117423
# 0.0001: 0.5546588896528468
# 1e-05: 0.5548408110536218
# w is integral
# w: 0.001, 0.5551303348248551, 0.0001: 0.5547
# w is fundu, 0.0001: 0.556040408488732
# w mom2 with round1: 1.0, 0.1: 0.5491515754493856, 0.001: 0.5559538615783635
# w mom2 with trunc4

1.0
1.0
-------------------------
0.40914053000293865
0.1
0.1
-------------------------
0.5504652364349819
0.01
0.01
-------------------------
0.556515761900757
0.001
0.001
-------------------------
0.5560161334686287
0.0001
0.0001
-------------------------
0.556040408488732
1e-05
1e-05
-------------------------
0.556040408488732


In [20]:
# c = [1.5,1.5,0.73,0.17,0.027,0.027] #[phi_coef,phi_coef,zdivrt_coef,zdivr_coef,xdivr_coef,ydivr_coef]
# min_samples_in_cluster = 1

# model = Clusterer()
# model.initialize(hits) 
# hits_with_dz = preprocess_hits(hits, 0)
# model.Hough_clustering(hits_with_dz,coef=c,epsilon=0.0048,min_samples=min_samples_in_cluster,
#                        n_loop=1000,verbose=True)

# submission = create_one_event_submission(0, hits, model.cluster)
# print('\n')

# score = score_event(truth, submission)
# print(score)

In [12]:
def Fun4BO(z_shift):
#     hits = get_hits()
    print(n_loop)
#     c = [w1,w1,w2,w3,w4,w5]
    c = [1.77,1.77,0.734,0.14,0.0195,0.025]
    model = Clusterer()
    model.initialize(hits) 
    hits_with_dz = preprocess_hits(hits, z_shift)
    min_samples_in_cluster = 1
    model.Hough_clustering(hits_with_dz,coef=c,epsilon=0.0048,min_samples=min_samples_in_cluster,
                       n_loop=370,verbose=True)

    submission = create_one_event_submission(0, hits, model.cluster)
    print('-------------------------')

    score = score_event(truth, submission)
    print(score)
    
    return score

In [13]:
# w1 = [0.001, 3.2] 
# w2 = [0.001, 3.2] 
# w3 = [0.001, 3.2] 
# w4 = [0.001, 3.2] 
# w5 = [0.001, 3.2] 
# w6 = [0.001, 3.2] 

# bo = BayesianOptimization(Fun4BO,pbounds = {'w1':w1, 'w2':w2, 'w3':w3, 'w4': w4, 'w5':w5})
# bo.maximize(init_points = 40, n_iter = 5, acq = "ucb", kappa = 2.576)

# for n_loop in np.arange(300, 460, 10):
#     print(n_loop)
#     Fun4BO(n_loop)
# w1: 1.77 (0.5450052751017225)
# w2: 0.734: 0.5453310726331103
# w3: 0.14: 0.5457328790148219 
# w4: 0.0195: 0.5493144276000794
# w5: 0.025: 0.5517975288706574
# n_loop: 370: 0.5548408110536218

In [14]:
def Fun4BO1(z_shift):
#     hits = get_hits()
    print(z_shift)
#     c = [w1,w1,w2,w3,w4,w5]
    c = [1.77,1.77,0.734,0.14,0.0195,0.025]
    model = Clusterer()
    model.initialize(hits) 
    hits_with_dz = preprocess_hits(hits, z_shift)
    min_samples_in_cluster = 1
    model.Hough_clustering(hits_with_dz,coef=c,epsilon=0.0048,min_samples=min_samples_in_cluster,
                       n_loop=370,verbose=True)

    return model.cluster

In [15]:
ls = []

l = Fun4BO1(0)
ls.append(l)

submission = create_one_event_submission(0, hits, l)
print('-------------------------')

score = score_event(truth, submission)
print(score)
    
for z_shift in np.arange(-5.5, -5.0, 0.01):
    print(z_shift)
    l = Fun4BO1(z_shift)
    ls.append(l)
    
    submission = create_one_event_submission(0, hits, l)
    print('-------------------------')

    score = score_event(truth, submission)
    print(score)

0
-------------------------
0.5548408110536218
-5.5
-5.5
-------------------------
0.4979654839813329
-5.49
-5.49
-------------------------
0.4137679016526512
-5.48
-5.48
-------------------------
0.3341160065633342
-5.470000000000001
-5.470000000000001
-------------------------
0.24724009794324278
-5.460000000000001
-5.460000000000001
-------------------------
0.16524925737396182
-5.450000000000001
-5.450000000000001
-------------------------
0.11265024162989001
-5.440000000000001
-5.440000000000001
-------------------------
0.07752435424025374
-5.4300000000000015
-5.4300000000000015
-------------------------
0.05259905731407198
-5.420000000000002
-5.420000000000002
-------------------------
0.03663954557608446
-5.410000000000002
-5.410000000000002
-------------------------
0.02543864330836862
-5.400000000000002
-5.400000000000002
-------------------------
0.017786123775768888
-5.390000000000002
-5.390000000000002
-------------------------
0.014817209413121311
-5.380000000000003
-5.38

In [17]:
import hdbscan
seed = 123
np.random.seed(seed)

def shift(l, n):
    return l[n:] + l[:n]

In [19]:

labels = np.zeros(num_hits,np.int32)
counts = np.zeros(num_hits,np.int32)

num_hits = len(hits)
lss = []
for i in range(len(ls)):
    labels1 = np.zeros(num_hits,np.int32)
    counts1 = np.zeros(num_hits,np.int32)
    ls1 = ls.copy()
    ls1 = shift(ls1, 1)
    np.random.shuffle(ls1)
    for l in ls1:
        c = make_counts(l)
        idx = np.where((c-counts>0) & (c<20))[0]
        labels1[idx] = l[idx] + labels1.max()
        counts1 = make_counts(labels1)
    l1 = labels1.copy()
    lss.append(l1)


labels = np.zeros(num_hits,np.int32)
counts = np.zeros(num_hits,np.int32)

for l in lss:
    c = make_counts(l)
    idx = np.where((c-counts>0) & (c<20))[0]
    labels[idx] = l[idx] + labels.max()
    counts = make_counts(labels)
    
    submission = create_one_event_submission(0, hits, labels)
    print('-------------------------')
    print('-------------------------')

    score = score_event(truth, submission)
    print(score)
    

-------------------------
-------------------------
0.003404480384503086
-------------------------
-------------------------
0.003477309274813337
-------------------------
-------------------------
0.008167709524794441
-------------------------
-------------------------
0.09337532294777887
-------------------------
-------------------------
0.08776587767388264
-------------------------
-------------------------
0.08342650437539689
-------------------------
-------------------------
0.08240054602102631
-------------------------
-------------------------
0.07966614256937776
-------------------------
-------------------------
0.076529576216016
-------------------------
-------------------------
0.07606311400402886
-------------------------
-------------------------
0.48944954441505506
-------------------------
-------------------------
0.48044408712669173
-------------------------
-------------------------
0.4802140455257118
-------------------------
-------------------------
0.4705229772

In [20]:
submission = create_one_event_submission(0, hits, labels)
print('-------------------------')
print('-------------------------')

score = score_event(truth, submission)
print(score)

-------------------------
-------------------------
0.485189209696906
